# Extract frames from a video


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

from hydra_zen import instantiate, builds
from buttermilk import BM
from buttermilk._core.config import Project
from promptflow.tracing import trace, start_trace
from rich import print as rprint
from hydra import initialize, compose

with initialize(version_base=None, config_path="../conf"):
    # Load config, specifying overrides for our particular job
    cfg = compose(config_name='config',
                  overrides=["+flows=frames"]
                  )
    
    Config = builds(Project, hydra_recursive=True)
    conf = instantiate(Config, **cfg)
    
    # print config
    rprint(conf)

# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=conf)


/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project(
    name='examples',
    job='testing',
    connections=[],
    secret_provider=CloudProviderCfg(
        type='azure',
        vault='https://suzorvault.vault.azure.net/',
        resource_group='rg-suzor_ai',
        models_secret='models'
    ),
    save_dest=CloudProviderCfg(type='gcp', project='dmrc-analysis', region='us-central1', bucket='dmrc-analysis'),
    logger='gcp',
    flows=[
        Flow(
            name='frame',
            num_runs=1,
            concurrency=20,
            agent=Agent(name='lc_dXnewS', concurrency=4, save_params=None),
            data=[],
            parameters={}
        )
    ],
    tracing=Tracing(enabled=True, endpoint=None, otlp_headers={}),
    verbose=False,
    cloud=[],
    run=RunCfg(platform='local', parameters={})
)

2024-11-08 14:18:46 J5HW6L4KT6 buttermilk buttermilk.py[ 224] INFO {'message': "Logging setup for: 20241108T0418Z-A3nL-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'examples', 'job': 'testing', 'task_id': '20241108T0418Z-A3nL-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/examples/testing/20241108T0418Z-A3nL-J5HW6L4KT6-suzor", 'project': 'examples', 'job': 'testing', 'run_id': '20241108T0418Z-A3nL-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/examples/testing/20241108T0418Z-A3nL-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
Prompt flow service has started...


In [4]:
from buttermilk._core.runner_types import Job, RecordInfo

record=RecordInfo(video='gs://dmrc-platforms/test/fyp/tiktok-imane-01.mp4')

job = Job(source="notebook", record=record)


In [6]:
record.as_langchain_message()


HumanMessage(content='')

In [ ]:
from buttermilk.flows.video.video import Analyst

flow_vars = cfg['run']['frame']
flow = Analyst(**flow_vars)
output = flow(content='see video', media_attachment_uri='gs://dmrc-platforms/test/fyp/tiktok-imane-01.mp4')

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output)


2024-08-28 21:04:20 26f087537f3a buttermilk buttermilk.py[ 227] INFO Logging setup for: {'function_name': 'default_project', 'job': 'development', 'logs': '20240828T1104Z-4NLQ-26f087537f3a-vscode', 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'default_project', 'job': 'development', 'task_id': '20240828T1104Z-4NLQ-26f087537f3a-vscode'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/default_project/development/20240828T1104Z-4NLQ-26f087537f3a-vscode
2024-08-28 21:04:20 26f087537f3a root _start_trace.py[  76] INFO collection: default_project
2024-08-28 21:04:20 26f087537f3a root _start_trace.py[  78] INFO resource attributes: {'service.name': 'promptflow', 'run_id': '20240828T1104Z-4NLQ-26f087537f3a-vscode', 'collection': 'default_project'}
2024-08-28 21:04:20 26f087537f3a root _start_trace.py[ 

In [ ]:

output = flow(content='see video', media_attachment_uri='gs://dmrc-platforms/test/fyp/tiktok-imane-02.mp4')

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output)


You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=default_project&uiTraceId=0x6c9823a1dec3c8a55bec2261dcf8c9ba
https://ai.azure.com/projecttrace/detail/0x6c9823a1dec3c8a55bec2261dcf8c9ba?wsid=/subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourceGroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod
You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=default_project&uiTraceId=0xa18d9b01bf4344f89093d53ff8233de2
https://ai.azure.com/projecttrace/detail/0xa18d9b01bf4344f89093d53ff8233de2?wsid=/subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourceGroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod
You can view the trace detail from the following URL:
http://127.0.0.1:23334/v1.0/ui/traces/?#collection=default_project&uiTraceId=0x56b578647f4c2760783cfc4f6674d92e
https://ai.azure.com/projecttrace/detail/0x56b578647f4c2

{   'framing': [   {   'blame_attribution': 'The speaker attributes blame to '
                                            "the woman who ignored Imane's "
                                            'attempt at comfort. This is '
                                            "evident in the phrase 'I can't "
                                            "get over how that woman ignored.' "
                                            'The language emphasizes the act '
                                            'of ignoring as unexpected and '
                                            'insensitive.',
                       'moral_evaluation': 'The statement implies a negative '
                                           'moral judgment of the woman who '
                                           'ignored the attempt at comfort.  '
                                           'The speaker frames ignoring '
                                           'someone offering comfort as '
          

In [ ]:

output = flow(content='see video', media_attachment_uri='gs://dmrc-platforms/test/fyp/tiktok-imane-03.mp4')

import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output)


{   'framing': [   {   'blame_attribution': 'Implicitly blames the situation '
                                            'or system that allowed the female '
                                            'boxer to be in a position where '
                                            'she feared for her safety.',
                       'moral_evaluation': 'Expresses concern for the female '
                                           "boxer's well-being and implies a "
                                           'sense of injustice or unfairness '
                                           'in the situation.',
                       'problem_definition': 'A talented female boxer quit a '
                                             'fight out of fear for her '
                                             'safety.',
                       'recommendation': 'While not explicitly stated, the '
                                         'text hints at a need for change to '
                     